In [5]:
import boto3
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import recall_score

## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'craig-shaffer-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to be read from s3 bucket
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# reading the datafile
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [6]:
#dropping observations w/ missing values
heart=heart.dropna()

In [7]:
#defining the input and target
x=heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
y=heart[['TenYearCHD']]

#splitting the data
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = .2,stratify=y)

# AdaBoost

In [10]:
#building the model
ada_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth=3),n_estimators=500).fit(x_train,y_train)

#predict on test
ada_pred = ada_md.predict_proba(x_test)[:,1]

#change likelihoods to labels
ada_labels = np.where(ada_pred<0.1,0,1)

print('the recall score of the AdaBoost Model is',recall_score(y_test,ada_labels))

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


the recall score of the AdaBoost Model is 1.0


# Random Forest

In [11]:
#building the model
rf_md = RandomForestClassifier(n_estimators=500,max_depth=3).fit(x_train,y_train)

#predict on test
rf_pred = rf_md.predict_proba(x_test)[:,1]

#change likelihoods to labels
rf_labels = np.where(rf_pred<0.1,0,1)

print('the recall score of the Random Forest Model is',recall_score(y_test,rf_labels))

/tmp/ipykernel_22155/2331082739.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_md = RandomForestClassifier(n_estimators=500,max_depth=3).fit(x_train,y_train)


the recall score of the Random Forest Model is 0.8660714285714286


Based on the results above, would want to use adaboost to predict likelihood of coronary heart disease becuase it had a higher recall score.